In [83]:
#sys env
import os
import sys
from dotenv import load_dotenv

#langchain
import langchain
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pypdf")
import pypdf


import chromadb

import pprint



import tiktoken

import pydantic
import lxml

import matplotlib.pyplot as plt

import logging


import zipfile
import requests

In [42]:
load_dotenv()
GEMINI_API=os.getenv("GEMINI_API")


In [24]:
#get documents from zip in pdf

logging.basicConfig(level=logging.INFO)

data_url = "https://storage.googleapis.com/benchmarks-artifacts/langchain-docs-benchmarking/cj.zip"
result = requests.get(data_url)
filename = "cj.zip"
with open(filename, "wb") as file:
   file.write(result.content)

with zipfile.ZipFile(filename, "r") as zip_ref:
   zip_ref.extractall()

In [75]:
folder_pdf="/Volumes/mashenka/delete/cyber/slides"

In [76]:
#load pdf functions
#learner guide - parse as image
#PP slides - parse pdf text only

def is_valid_pdf(file_path):
    try:
        with open(file_path, 'rb') as f:
            header = f.read(5)
            return header == b'%PDF-'
    except Exception:
        return False


loader=PyPDFDirectoryLoader(folder_pdf)
slides=loader.load()


In [84]:
print(len(slides))
pprint.pp(slides[24].page_content)

72
'VU23215\nTest concepts and procedures for cyber security\n1\nSession 2'


In [71]:
file_path = "/Volumes/mashenka/Python312/cj/cj.pdf"
loader = PyPDFLoader(file_path, extract_images=True)
pages = []
async for page in loader.alazy_load():
    pages.append(page)




{'source': '/Volumes/mashenka/Python312/cj/cj.pdf', 'page': 0, 'page_label': '1'}


In [68]:
#load pdfs
folder_path = "/Volumes/mashenka/Python312/cj"

loader = PyPDFDirectoryLoader(folder_path, extract_images=True)
all_pdf=loader.lazy_load()


def is_valid_pdf(file_path):
    try:
        with open(file_path, 'rb') as f:
            header = f.read(5)
            return header == b'%PDF-'
    except Exception:
        return False

def load_pdfs(path: str):
    all_pdf = []
    for filename in os.listdir(path):
        file_path=os.path.join(path,filename)
        if filename.endswith(".pdf") and is_valid_pdf(file_path):
            file_path = os.path.join(path, filename)
            try:
                #extract_images requires pip install rapidocr-onnxruntime
                loader = PyPDFLoader(file_path, extract_images=True)
                doc = loader.load()
                texts = [d.page_content for d in doc]
                all_pdf.append(texts)

                #extracting images as image files
                for page in doc:
                    for image in doc.images:
                        image_filename=f"{filename}_page_{page.page_number}_image_{image["]}"

            except Exception as e:
                print(f"Failed to process file {filename}: {e}")
    return all_pdf

pdfs=load_pdfs(folder_path)


## Load images and summarize images to be further used as text

In [ ]:
from transformers import BLIP